### Importação das bibliotecas

<b>Numpy</b> - Computação numérica

In [1]:
import numpy as np

<b>Lasso</b> - É um método de regressão que realiza tanto seleção de variáveis como regularização para aprimorar a previsão e interpertabilidade do modelo estatístico produzido.

<b>SVR</b> - Regressão por Vetores de Suporte é um tipo de Máquina de Vetores de Suporte especializado para problemas de regressão e tem como objetivo encontrar um função que produza saídas contínuas para os dados de treinamento que se aproximem ao máximo de seu rótulo desejado [Faceli et al. 2011].

<b>MLP</b> -  Este tipo de modelo inteligente se encaixa na classificação de método baseado em otimização, pois o treinamento deste modelo consiste em minimizar (ou maximizar) uma função objetivo. No caso de problemas supervisionados, o valor dos elementos que se quer descobrir é considerado na formulação realizada. As Redes Neurais Artificiais (RNA) são baseadas no cérebro humano e são sistemas computacionais distribuídos compostos por unidades de processamento simples, essas unidades, conhecidas como neurônios artificiais, computam funções matemáticas [Faceli et al. 2011].

<b>Random Forest Regressor</b> - Uma Floresta Aleatória é um estimador que ajusta um númerp de Árvores de Decisão classificadores em várias sub-amostras do dataset e usa a média para melhorar a precisão e controle de sobreajuste.

<b> Gradient Boosting Regressor</b> - É um método de otimização de funções de perda diferenciáveis arbitrárias. Em cada etapa uma Árvore de Decisão é treinada no negativo do gradiente dado pela função de perda.

In [2]:
from sklearn.linear_model import Lasso 
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

<b>GridSearchCV</b>     - Busca em Grade é um processo de treinamento de avaliação de modelos para encontrar uma configuração ótima de parâmetros para determinado modelo.

<b>KFold</b>            - Método de treinamento e avaliação dos modelos

<b>sklearn.metrics</b>  - Métricas de avaliação de modelo MAE, MSE e R2

<b>sqrt</b>             - Função do cálculo da Raiz Quadrada para geração da métrica RMSE

<b>ceil</b>             - Função para arrendodamento para cima

In [3]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from math import sqrt, ceil

In [4]:
import warnings
warnings.simplefilter("ignore")

In [5]:
def evaluate_model(model):
    kf = KFold(n_splits=10, shuffle=True)

    kf.get_n_splits(X)

    mae = []
    rmse = []
    r2 = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        mae.append(mean_absolute_error(y_test, y_pred))
        rmse.append(sqrt(mean_squared_error(y_test, y_pred)))
        r2.append(r2_score(y_test, y_pred))

    print('A média da métrica MAE foi de {}'.format(round(np.mean(mae), 2)))
    print('A média da métrica RMSE foi de {}'.format(round(np.mean(rmse), 2)))
    print('A média da métrica R2 foi de {}'.format(round(np.mean(r2), 4)))

Heurística da Pirâmide para geração das possíveis melhores configurações de neurônios de uma rede neural artificial de 2 camadas ocultas, baseada num <i>alpha</i>, número de dados de entrada e número de saídas.

In [6]:
def piramid(alpha=2, n_in=7, n_out=1):
    neurons = int(ceil(alpha * sqrt(n_in * n_out)))
    ret = [(neurons,)]
    ret += [(neurons-n, n) for n in range(1, neurons)]
    return ret

In [7]:
def grid_search(model, params, X, y):
    regressor = GridSearchCV(model, params, scoring='neg_mean_absolute_error', n_jobs=-1, verbose=10)
    regressor.fit(X, y)
    print(np.abs(regressor.best_score_))
    print(regressor.best_params_)

In [8]:
# Carregamento dos atributos preditores e alvo
X = np.load('dataset/X_processed.npy')
y = np.load('dataset/y.npy')

In [9]:
X.shape

(1460, 163)

In [10]:
# Número de atributos preditores
n_features = X.shape[1]

### Definição dos parâmetros a serem considerados durante a Busca em Grade

In [11]:
# Parâmetros e hiperparâmetros a serem considerados para o treinamento da Rede Neural Artificial
mlp_params = {
    'activation': ['logistic', 'relu', 'tanh'], 
    'max_iter': [100, 150, 200],
    'hidden_layer_sizes': list(piramid(alpha=0.5, n_in=n_features) + 
                               piramid(alpha=2, n_in=n_features) + 
                               piramid(alpha=3, n_in=n_features))
}

In [12]:
# Parâmetros e hiperparâmetros a serem considerados para o treinamento do Lasso
lasso_params = {'alpha': [0.5, 1.0, 2.0],
                'max_iter': [800, 900, 1000]}

In [13]:
# Parâmetros e hiperparâmetros a serem considerados para o treinamento do Regressor por Vetores de Suporte
svr_params = {'C': [0.5, 1.0, 2.0],
       'kernel': ['linear','poly','rbf'],
       'max_iter': [800, 900, 1000]}

In [14]:
# Parâmetros e hiperparâmetros a serem considerados para o treinamento da Floresta Aleatória
rf_params = {'n_estimators': [500, 1000, 1500],
             'max_features': ['auto', 'sqrt', 'log2']}

In [15]:
# Parâmetros e hiperparâmetros a serem considerados para o treinamento do Gradient Boosting
gb_params = {'n_estimators': [500, 1000, 1500],
             'learning_rate': [0.05, 0.1],
             'max_features': ['auto', 'sqrt']}

### Busca em Grade a partir dos modelos

In [16]:
lasso = Lasso()
grid_search(lasso, lasso_params, X, y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done  35 out of  45 | elapsed:    6.0s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done  40 out of  45 | elapsed:    6.4s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    6.8s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    6.8s finished


19753.50258281688
{'alpha': 2.0, 'max_iter': 800}


In [17]:
svr = SVR()
grid_search(svr, svr_params, X, y)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:   12.0s
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:   15.0s finished


29712.267633708834
{'C': 0.5, 'kernel': 'linear', 'max_iter': 1000}


In [18]:
rf = RandomForestRegressor(criterion='mae')
grid_search(rf, rf_params, X, y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 10.0min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 12.6min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 15.6min
[Parallel(n_jobs=-1)]: Done  35 out of  45 | elapsed: 16.6min remaining:  4.8min
[Parallel(n_jobs=-1)]: Done  40 out of  45 | elapsed: 17.4min remaining:  2.2min
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed: 18.5min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed: 18.5min finished


17484.164752739725
{'max_features': 'sqrt', 'n_estimators': 500}


In [19]:
gb = GradientBoostingRegressor(criterion='mae', loss='huber')
grid_search(gb, gb_params, X, y)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 10.1min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 13.3min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 16.4min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 22.8min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed: 31.2min
[Parallel(n_jobs=-1)]: Done  52 out of  60 | elapsed: 33.8min remaining:  5.2min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 38.2min finished


15907.42306737458
{'learning_rate': 0.05, 'max_features': 'sqrt', 'n_estimators': 1000}


In [20]:
mlp = MLPRegressor(solver='adam')
grid_search(mlp, mlp_params, X, y)

Fitting 5 folds for each of 648 candidates, totalling 3240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:   15.0s
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:   17.9s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   21.1s
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:   24.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   

30806.29622113934
{'activation': 'relu', 'hidden_layer_sizes': (17, 22), 'max_iter': 200}


Com os melhores parâmetros encontrados a partir da Busca em Grade, serão criados modelos a partir desses parâmetros para realizar mais treinamentos e testes.

Os modelos que apresentaram os melhores resultados foram a Floresta Aleatória Regressora e Gradient Boosting com os seguintes parâmetros:

Floresta Aleatória: max_features='sqrt', n_estimators=1500 e criterion='mae'

Gradient Boosting:learning_rate=0.05, max_features='sqrt', n_estimators=1500, criterion='mae' e loss='huber'

### Treinamento e avaliação dos melhores modelos a partir do K-fold e das métricas MAE, RMSE e R2

In [21]:
best_rf = RandomForestRegressor(max_features= 'sqrt', 
                                n_estimators= 500,
                                criterion='mae')

In [22]:
evaluate_model(best_rf)

A média da métrica MAE foi de 17319.5
A média da métrica RMSE foi de 29203.91
A média da métrica R2 foi de 0.8621


In [23]:
best_gb = GradientBoostingRegressor(learning_rate= 0.05, 
                                    max_features= 'sqrt',
                                    n_estimators= 1000, 
                                    criterion= 'mae',
                                    loss='huber')

In [24]:
evaluate_model(best_gb)

A média da métrica MAE foi de 16143.85
A média da métrica RMSE foi de 26795.49
A média da métrica R2 foi de 0.8807


O Gradient Boosting foi o modelo inteligente que apresentou os melhores resultados e será analisado mais detalhadamente.